# Assignment 2: Object Detection for EE 298Z

Kaggle notebook version for running the code in [this repository](https://github.com/amielle/ee298z-hw2-object-detection). Run the cells in the proper order and ensure that step 1 is followed before running sections 2-4.

## Table of Contents
1. Setup prerequisites
2. Run test.py
3. Run train.py
4. Plot predictions on sample images

## 1. Setup prerequisites

Install libraries not yet present in the Kaggle environment and update torchvision and torch.

In [1]:
!pip install gdown
!pip install torchvision -U
!pip install torch -U
!pip install pycocotools

Clone the source repository.

In [2]:
!git clone https://github.com/amielle/ee298z-hw2-object-detection

Change the directory to the code repo to access the scripts and modules.

In [3]:
import os
os.chdir("./ee298z-hw2-object-detection")

## 2. Run test.py

Uses the pre-trained model from the repo author to evaluate the model across the test dataset.

In [4]:
!python test.py

## 3. Run train.py

Allows the user to train their own model and displays test metrics at the end. There may be some minor discrepancies from the original pre-trained model due to some differences in the initialization of weights and others.

In [5]:
!python train.py

## 4. Plot predictions on sample images

Loads the original pre-trained model from the repo author. Plots sample bounding boxes and predicted classes (numerical values converted to equivalent text) for sample images on the test dataset.

In [6]:
from torchvision import transforms
from torch.utils.data import DataLoader
import torch

import odutils.label_utils as label_utils
import odutils.dataprep as dataprep
import odutils.odmodel as odmodel
from odutils.config import return_config

import detection.utils as utils
from detection.engine import train_one_epoch, evaluate

main_dir = os.getcwd()

# Download dataset and pre-trained model
dataprep.setup_files(dataset_filename="drinks.tar.gz", 
                     unzip_dataset=True)

config = return_config(main_dir)

test_dict, test_classes = label_utils.build_label_dictionary(
config['test_split'])
test_split = odmodel.ImageDataset(test_dict, test_classes, transforms.ToTensor())

test_loader = DataLoader(test_split,
                         batch_size=config['batch_size'],
                         shuffle=False,
                         num_workers=config['num_workers'],
                         pin_memory=config['pin_memory'],
                         collate_fn=utils.collate_fn)

# Change as needed
model_path = f"{main_dir}/adulay-fasterrcnn_resnet50_fpn-1651304089.3776634.pth"
model = odmodel.load_model(od_trained_model=model_path)

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# Plotting part
for filename in list(test_dict.keys())[:10]:
    odmodel.detect_drinks(model, 
                          filename=filename, 
                          detection_threshold=0.9)